In [ ]:
import os
import re
import numpy as np
import pandas as pd

In [ ]:
# path_settings
try:
    from google.colab import drive
    base_dir ='content/'
    data_path='/'.join([base_dir,'data'])
    
except ModuleNotFoundError:
    base_dir = '/Users/jack/Desktop/things/projects/samsung_material_discovery'
    data_path='/'.join([base_dir,"data"])

In [ ]:
# load csv data to dataframe
df_train=pd.read_csv("/".join([data_path,"train_set.ReorgE.csv"]),index_col=0)
df_test=pd.read_csv("/".join([data_path,"test_set.csv"]), index_col=0)

In [ ]:
# make preprocessed_data folder
save_path = '/'.join([base_dir,"preprocessed_data"])

if not os.path.isdir(save_path):
    os.mkdir(save_path)
    os.mkdir('/'.join([save_path,"train_mol"]))
    os.mkdir('/'.join([save_path,"test_mol"]))

# slice smiles data and target
df_train.iloc[:,-2:].to_csv("/".join([base_dir,'preprocessed_data/train_target.csv']))
df_train.iloc[:,:1].to_csv("/".join([base_dir,'preprocessed_data/train_smile.csv']))
df_test.iloc[:,:1].to_csv("/".join([base_dir,'preprocessed_data/test_smile.csv']))

### mol file preprocessing 

In [ ]:
def read_mol(path):
    mol_data = open(file=path,mode='r').read().split('\n')
    return mol_data

def get_count(mol_data):
    result= re.sub(' +', ' ', mol_data[3].lstrip()).split(' ')[:2]
    return result

def get_location(mol_data):
    li_location = list()
    for _ in mol_data:
        _ = _.lstrip()
        _=re.sub(' +',' ',_).split(' ')
        if len(_) == 16:
            li_location.append(_[:4])
    return li_location

def get_bond(mol_data):
    li_bond = list()
    for _ in mol_data:
        _ = _.lstrip()
        _=re.sub(' +',' ',_).split(' ')
        if len(_) == 4:
            li_bond.append(_[:3])
    return li_bond

def mol2data(file_path):
    mol_data=read_mol(file_path)
    
    count=np.array(get_count(mol_data)).reshape(1,2)
    location=np.array(get_location(mol_data))
    bond=np.array(get_bond(mol_data))
    
    return count, location, bond

def save_data(data,save_path):
    np.save(save_path,data)
    return None

In [ ]:
print(save_path)
print(data_path)

/Users/jack/Desktop/things/projects/samsung_material_discovery/preprocessed_data
/Users/jack/Desktop/things/projects/samsung_material_discovery/data


In [ ]:
# test_mol_file -> 3_npy (count,location,bond)
for file_name in df_test.index:
    for status in ['ex','g']:
        file_path="/".join([data_path,'mol_files/test_set/{0}_{1}.mol'.format(file_name,status)])
        count, location, bond = mol2data(file_path)

        path_count = "/".join([save_path,'test_mol/{0}_count'.format(file_name)])
        path_location = "/".join([save_path,'test_mol/{0}_{1}_location'.format(file_name,status)])
        path_bond = "/".join([save_path,'test_mol/{0}_{1}_bond'.format(file_name,status)])

        np.save(path_count,count)
        np.save(path_location,location)
        np.save(path_bond,bond)
        

# train_mol_file -> 3_npy (count,location,bond)
for file_name in df_train.index:
    for status in ['ex','g']:
        file_path="/".join([data_path,'mol_files/train_set/{0}_{1}.mol'.format(file_name,status)])
        count, location, bond = mol2data(file_path)

        path_count = "/".join([save_path,'train_mol/{0}_count'.format(file_name)])
        path_location = "/".join([save_path,'train_mol/{0}_{1}_location'.format(file_name,status)])
        path_bond = "/".join([save_path,'train_mol/{0}_{1}_bond'.format(file_name,status)])

        np.save(path_count,count)
        np.save(path_location,location)
        np.save(path_bond,bond)